In [24]:
import numpy as np

# 定义作为拟合目标的函数(sinx + x)
class fun:
    def __init__(self, noise_level=0.0):
        '''noise_level为噪声强度，默认无噪声'''
        self.noise_level = noise_level

    def __call__(self, x):
        y = np.sin(x)
        if self.noise_level > 0:
            noise = np.random.normal(0, self.noise_level, size=x.shape)
            y += noise
        return y

In [6]:
def generate_data(domain, noise_level=0.0, num_train=1000, num_test=200):
    """
    生成训练数据和测试数据。
    
    :param domain: 定义域，即 x 的范围，元组类型 (x_min, x_max)
    :param noise_level: 噪声强度，默认为 0，即不添加噪声
    :param num_train: 训练集的样本数
    :param num_test: 测试集的样本数
    :return: 返回训练数据和测试数据
    """
    # 创建 函数 类的实例
    func = fun(noise_level=noise_level)
    
    # 生成训练集
    x_train = np.random.uniform(domain[0], domain[1], num_train)
    y_train = func(x_train)
    
    # 生成测试集
    x_test = np.random.uniform(domain[0], domain[1], num_test)
    y_test = func(x_test)
    
    return (x_train, y_train), (x_test, y_test)

In [26]:
class SimpleNN:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        """
        初始化神经网络的结构和参数。
        :param input_size: 输入层的大小
        :param hidden_size1: 第一隐藏层的大小
        :param hidden_size2: 第二隐藏层的大小
        :param output_size: 输出层的大小
        """
        # 初始化权重和偏置
        self.W1 = np.random.randn(input_size, hidden_size1) * 0.01
        self.b1 = np.zeros((1, hidden_size1))
        self.W2 = np.random.randn(hidden_size1, hidden_size2) * 0.01
        self.b2 = np.zeros((1, hidden_size2))
        self.W3 = np.random.randn(hidden_size2, output_size) * 0.01
        self.b3 = np.zeros((1, output_size))

    def relu(self, x):
        """ReLU 激活函数"""
        return np.maximum(0, x)

    def relu_derivative(self, x):
        """ReLU 激活函数的导数"""
        return (x > 0).astype(float)

    def forward(self, x):
        """前向传播"""
        self.z1 = np.dot(x, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.relu(self.z2)
        self.z3 = np.dot(self.a2, self.W3) + self.b3
        return self.z3

    def backward(self, x, y, learning_rate=0.01):
        """反向传播，计算梯度并更新权重"""
        m = x.shape[0]

        # 计算损失函数关于输出的梯度
        dz3 = self.z3 - y
        dW3 = np.dot(self.a2.T, dz3) / m
        db3 = np.sum(dz3, axis=0, keepdims=True) / m

        # 计算损失函数关于隐藏层2的梯度
        dz2 = np.dot(dz3, self.W3.T) * self.relu_derivative(self.z2)
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m

        # 计算损失函数关于隐藏层1的梯度
        dz1 = np.dot(dz2, self.W2.T) * self.relu_derivative(self.z1)
        dW1 = np.dot(x.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m

        # 更新权重和偏置
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W3 -= learning_rate * dW3
        self.b3 -= learning_rate * db3

    def train(self, x_train, y_train, epochs=1000, learning_rate=0.01):
        """训练神经网络"""
        for epoch in range(epochs):
            for i in range(x_train.shape[0]):
                x = x_train[i]
                y = y_train[i]
                y_pred = self.forward(x)
    
                # 反向传播
                self.backward(x, y, learning_rate)

            # 每100次打印一次损失
            if epoch % 100 == 0:
                # 使用 RMSE 作为损失函数
                loss = np.sqrt(np.mean((y_pred - y_train) ** 2))  # 均方根误差
                print(f"Epoch {epoch}, Loss (RMSE): {loss}")

    def predict(self, x):
        """用于推理"""
        return self.forward(x)

In [28]:
domain = (0, 10)
noise_level = 0.0
(x_train, y_train), (x_test, y_test) = generate_data(domain, noise_level)
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape(-1, 1)
x_test = x_test.reshape(-1, 1)
# print(x_train.shape)
# print(x_test.shape)

# 定义神经网络
input_size = 1
hidden_size1 = 64  # 第一隐藏层大小
hidden_size2 = 32  # 第二隐藏层大小
output_size = 1
model = SimpleNN(input_size, hidden_size1, hidden_size2, output_size)
model.train(x_train, y_train, epochs=1000, learning_rate=0.0001)

# 测试神经网络的表现
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

train_loss = np.mean((y_pred_train - y_train) ** 2)
test_loss = np.mean((y_pred_test - y_test) ** 2)

print(f"Training Loss: {train_loss}")
print(f"Test Loss: {test_loss}")

(1000,)
(200,)
Epoch 0, Loss (RMSE): 0.6830415783686106
Epoch 100, Loss (RMSE): 0.6629739890664503
Epoch 200, Loss (RMSE): 0.6629739590407072
Epoch 300, Loss (RMSE): 0.6629739377333441
Epoch 400, Loss (RMSE): 0.6629739164995102
Epoch 500, Loss (RMSE): 0.6629738952660054
Epoch 600, Loss (RMSE): 0.6629738739739166
Epoch 700, Loss (RMSE): 0.6629738525789425
Epoch 800, Loss (RMSE): 0.6629738310390203
Epoch 900, Loss (RMSE): 0.662973809330617
Training Loss: 0.4395345545677529
Test Loss: 0.5028833938998686
